In [1]:
import os, sys
import yfinance as yf
# import ta
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import date, timedelta, datetime
from itertools import product
from IPython.display import clear_output
import matplotlib.pyplot as mp
from csv import writer
import warnings

/home/diti/miniconda3/envs/devbox/lib/python3.12/site-packages/telegram/utils/request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(


In [2]:
warnings.filterwarnings('ignore')

Strategies

RSI and Moving Average Crossover Strategy Indicators: \
1. RSI (21), 5 Exponential Moving Average (EMA), 12 Exponential Moving Average (EMA).
2. Entry Rules:Buy Signal: When the 5EMA crosses above the 12EMA and the RSI crosses above the 50 level.
3. Sell Signal: When the 5EMA crosses below the 12EMA and the RSI crosses below the 50 level.
4. Stop Loss/Take Profit: Set a stop loss of 15 pips and a take profit of 30 pips.
5. Example: If you are trading EUR/USD on a 30-minute chart and the 5EMA crosses above the 12EMA while the RSI crosses above 50, you would enter a buy position. Conversely, if the 5EMA crosses below the 12EMA and the RSI crosses below 50, you would enter a sell position .

In [67]:
def execute_main():
    # dt_range = pd.date_range('2019-12-31', '2020-12-31', freq='6m') 
    # define start and end date
    end_date = datetime.now()-timedelta(days=30)
    start_date = end_date - timedelta(days=390)

    # extract data from yahoo finanace
    btc_price = vbt.YFData.download(
    ["BTC-USD", "ETH-USD"], #"XMR-USD", "ADA-USD"],
    interval='1d',
    start = start_date,
    end = end_date,
    missing_index='drop').get("Close")

    # Initialize a Indicator Factory using custom indicator
    # Step2. Receipe how to create an indicator
    ind = vbt.IndicatorFactory(
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window", "ma_window"],
        output_names = ["value"]
    ).from_apply_func(
        custom_indicator,
        rsi_window=[14, 20],
        ma_window=[50, 60]
    )

    # Step3. execute the custom indicator with the given parameters
    res = ind.run(
        btc_price,
        rsi_window=[10, 20, 30],
        ma_window=[50, 60, 70]
    )

    return res

    # Results
    for _col in res.value.columns:
        print("-------------------{}------------------------------".format(_col))
        _asset = res.value[_col].to_numpy()
        _entry = np.where(_asset==1, True, False)
        _exit = np.where(_asset==-1, True, False)
        
        pf = vbt.Portfolio.from_signals(btc_price[_col[2]], _entry, _exit)
        print(pf.stats())
        pf.plot(title=_col[2]).show()
        print("----------------------------------------------------------")

In [68]:
# Step 1. Create a custom indicator
def custom_indicator(close, rsi_window=14, ma_window=50):
    rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    # we will create a signal
    # For example rsi between 35 and 70, and price will be above ma,we will buy
    trend = np.where(rsi>70, -1, 0)
    trend = np.where((rsi<30) & (close < ma), 1, trend)
    return trend

In [69]:
res  = execute_main()

In [70]:
btc_usd_entries = res.value[10][50]['BTC-USD'][res.value[10][50]['BTC-USD']==1]


In [71]:
btc_usd_entries

Date
2024-01-18 00:00:00+00:00    1
2024-01-19 00:00:00+00:00    1
2024-01-20 00:00:00+00:00    1
2024-01-21 00:00:00+00:00    1
2024-01-22 00:00:00+00:00    1
2024-01-23 00:00:00+00:00    1
2024-01-25 00:00:00+00:00    1
2024-04-17 00:00:00+00:00    1
2024-04-18 00:00:00+00:00    1
2024-04-30 00:00:00+00:00    1
2024-05-01 00:00:00+00:00    1
2024-05-02 00:00:00+00:00    1
2024-06-14 00:00:00+00:00    1
2024-06-18 00:00:00+00:00    1
2024-06-19 00:00:00+00:00    1
2024-06-20 00:00:00+00:00    1
2024-06-21 00:00:00+00:00    1
2024-06-22 00:00:00+00:00    1
2024-06-23 00:00:00+00:00    1
2024-06-24 00:00:00+00:00    1
2024-06-25 00:00:00+00:00    1
2024-06-26 00:00:00+00:00    1
2024-06-27 00:00:00+00:00    1
2024-06-28 00:00:00+00:00    1
2024-06-29 00:00:00+00:00    1
2024-07-05 00:00:00+00:00    1
2024-07-07 00:00:00+00:00    1
2024-07-11 00:00:00+00:00    1
2024-08-04 00:00:00+00:00    1
2024-08-05 00:00:00+00:00    1
2024-08-06 00:00:00+00:00    1
2024-08-07 00:00:00+00:00    1
202

In [77]:
df = pd.DataFrame({'a':[1,2,3,4], 'b':[3,5,6,7]})

In [78]:
df

,a,b
0,1,3
1,2,5
2,3,6
3,4,7
